In [1]:
%load_ext line_profiler
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import old.tools, old.RNTK_avg
import jax
import symjax
import symjax.tensor as T
import copy
from RNTK_NEW import RNTK
import time

In [3]:
dataset = "trains"
# dataset = "bank"

In [4]:
dic = {k:v for k,v in map(lambda x : x.split(), open("data" + "/" + dataset + "/" + dataset + ".txt", "r").readlines())}

In [5]:
# N = int(dic["n_patrons1="])
# length = int(dic["n_entradas="])
N = 1000
length = 100

In [6]:
DATA = T.Placeholder((N, length), 'float32', name = "X")
x = DATA[:,0]
X = x*x[:, None]
n = X.shape[0]

In [64]:
from RNTK_onediag import RNTK as RNTKOD
rntkod = RNTKOD(dic, X, n) #could be flipped

In [68]:
lin_ema = rntkod.create_func_for_diag()

In [83]:
%%timeit
# start = time.time()
lin_ema = rntkod.create_func_for_diag()
# print(time.time() - start)
# start = time.time()
final_ema = rntkod.get_ends_of_diags(lin_ema)
# print(time.time() - start)
# start = time.time()
rntk_func = symjax.function(DATA, outputs=final_ema)
# print(time.time() - start)

6.587394952774048
0.8630499839782715
0.00037407875061035156


In [ ]:
for t in range(0, Ti):
    for l in range(0, L):
        print(l, t)
        if ((t == l) & (l == 0)):
            #assuming ||x|| means the norm of that vector
            qmatrix[l, t] = su**2 * normv(X[0])**2 + sb**2 + sh ** 2

        if ((t == 0) & (l > t)):

            S,D = VTq(capQ(qmatrix[l-1,t]))
            qmatrix[l, t] = su**2 * S + sb**2

        if ((t > l) & (l == 0)):
            S,D = VTq(capQ(qmatrix[l,t-1]))
            qmatrix[l, t] = sw**2 * S + su**2 * normv(X[t])**2 + sb**2

        if ((t > l) & (l > 0)):
            S_left, D_left = VTq(capQ(qmatrix[l,t-1]))
            S_right, D_right = VTq(capQ(qmatrix[l-1,t]))
            qmatrix[l, t] = sw**2 * S_left + su**2 * D_right + sb**2

        print(qmatrix[l, t])


In [9]:
fake_data = np.zeros((N,length))

In [11]:
start = time.time()
result = rntk_func(fake_data)
gp_kernel = result[0]
rntk_kernel = result[1]
print(time.time() - start)

0.05171513557434082


In [12]:
gp_kernel.shape

(3, 3, 1000, 1000)